In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas
import math
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import (Dense,LSTM,Concatenate,Layer,Lambda,
                          Input,Multiply,SimpleRNN,GRU,
                          TimeDistributed)
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


In [2]:
from os.path import join as pjoin
DATA_DIR = pjoin('..', 'data')
MAX_CSV = pjoin(DATA_DIR, 'MaxV_10.csv')
DAT_CSV = pjoin(DATA_DIR, 'Data319.csv')

dataframeMV = pandas.read_csv(MAX_CSV, header=None, engine='python')
datasetMV = dataframeMV.values
datasetMV = datasetMV.astype('float32')
maxA = datasetMV[0,:]
maxT = datasetMV[1,:]

In [3]:
def create_dataset(ns):
    dataX, dataY = [], []
    maxD = 0
    minD = 10000
    for c in ns:
        dataframe = pandas.read_csv(DAT_CSV, usecols=[int(c)], engine='python')
        dataframe.dropna(how='all',inplace=True)
        dataset = dataframe.values
        dataset = dataset.astype('float32')
        sql = len(dataset)
        if np.max(dataset) > maxD:
            maxD = np.max(dataset)
        if np.min(dataset) < minD:
            minD = np.min(dataset)
        a = dataset[:sql-1,0]
        b = dataset[1:sql,0]
        #a = np.reshape(a,(sql-1,1))
        #b = np.reshape(b,(sql-1,1))
        dataX.append(a)
        dataY.append(b)
    return np.array(dataX), np.array(dataY), minD, maxD

def create_dataset_mscale(ns,maxDV,minD):
    dataX, dataY = [], []
    for c in ns:
        #print(c)
        dn = int(c/5)
        #print(dn)
        maxD = maxDV[dn]
        dataframe = pandas.read_csv(DAT_CSV, usecols=[int(c)], engine='python')
        dataframe.dropna(how='all',inplace=True)
        dataset = dataframe.values
        dataset = dataset.astype('float32')
        dataset = (dataset - minD)/(maxD-minD)
        sql = len(dataset)
        a = dataset[:sql-1,0]
        b = dataset[1:sql,0]
        #a = np.reshape(a,(sql-1,1))
        #b = np.reshape(b,(sql-1,1))
        dataX.append(a)
        dataY.append(b)
    return np.array(dataX), np.array(dataY)

In [4]:
ActiveLoc = np.asarray(np.linspace(0,235,48))
#ActiveLoc = [200]
TotalLoc = np.asarray(ActiveLoc) + 1
PopLoc = np.asarray(ActiveLoc) + 3
MedLoc = np.asarray(ActiveLoc) + 4

In [5]:
lb = 5

trainActive, targetActive = create_dataset_mscale(ActiveLoc,maxA,0)

trainTotal, targetTotal = create_dataset_mscale(TotalLoc,maxT,0)

trainPop, targetPop, minPop, maxPop = create_dataset(PopLoc)
#maxPop = (np.max(targetPop))
#minPop = np.min(trainPop)
trainPop = (trainPop - minPop)/(maxPop - minPop)
targetPop = (targetPop - minPop)/(maxPop - minPop)

trainMed, targetMed, minMed, maxMed = create_dataset(MedLoc)
#maxMed = np.max(targetMed)
#minMed = np.min(trainMed)
trainMed = (trainMed - minMed)/(maxMed - minMed)
targetMed = (targetMed - minMed)/(maxMed - minMed)

In [6]:
AllInput = Input(shape=(None,4))
RecInput = Lambda(lambda AllInput:AllInput[:,:,:2],name='RecIn')(AllInput)
HidInput = Lambda(lambda AllInput:AllInput[:,-1:,2:],name='HIn')(AllInput)
H = LSTM(
    20, input_shape=(None,2), return_sequences=True
)(RecInput)

D2 = Dense(20,activation='softmax')(HidInput)
# Duplicate for each timestep before concatenate
D2 = Lambda(
    lambda x: tf.tile(x, (1, tf.shape(H)[1], 1)),
    name='tile'
)(D2)
Data = Concatenate()([H, D2])
out = TimeDistributed(Dense(2,activation='relu'))(Data)

CoVid = Model(AllInput,out)
CoVid.summary()
CoVid.compile(loss='mean_squared_error', optimizer='adam')

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 4)      0                                            
__________________________________________________________________________________________________
HIn (Lambda)                    (None, None, 2)      0           input_1[0][0]                    
__________________________________________________________________________________________________
RecIn (Lambda)                  (None, None, 2)      0           input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, None, 20)     60          HIn[0][0]                        
____________________________________________________________________________________________

In [7]:
# Loop through each training curve from different regions
# Treat each region as a single batch... 
# Reformat into trainX (1,time_steps,features) and trainY  (1, time_steps,features)

for r in range(48):
    print('Epoch:', r)
    f1 = np.reshape(trainActive[0],(1,trainActive[0].shape[0],1))
    f2 = np.reshape(trainTotal[0],(1,trainTotal[0].shape[0],1))
    f3 = np.reshape(trainPop[0],(1,trainPop[0].shape[0],1))
    f4 = np.reshape(trainMed[0],(1,trainMed[0].shape[0],1))
    trainX = np.concatenate((f1,f2,f3,f4),axis=2)

    t1 = np.reshape(targetActive[0],(1, targetActive[0].shape[0],1))
    t2 = np.reshape(targetTotal[0],(1, targetTotal[0].shape[0],1))
    trainY = np.concatenate((t1,t2),axis=2)
    
    CoVid.train_on_batch(trainX, trainY)
    print(CoVid.evaluate(trainX, trainY))

Epoch: 0
1/1 [==============================] - 0s 49ms/step
0.25505560636520386
Epoch: 1
1/1 [==============================] - 0s 4ms/step
0.24671287834644318
Epoch: 2
1/1 [==============================] - 0s 3ms/step
0.23851928114891052
Epoch: 3
1/1 [==============================] - 0s 3ms/step
0.23050709068775177
Epoch: 4
1/1 [==============================] - 0s 3ms/step
0.22269907593727112
Epoch: 5
1/1 [==============================] - 0s 3ms/step
0.2151135504245758
Epoch: 6
1/1 [==============================] - 0s 2ms/step
0.20777080953121185
Epoch: 7
1/1 [==============================] - 0s 2ms/step
0.20069456100463867
Epoch: 8
1/1 [==============================] - 0s 2ms/step
0.19391290843486786
Epoch: 9
1/1 [==============================] - 0s 3ms/step
0.18745926022529602
Epoch: 10
1/1 [==============================] - 0s 2ms/step
0.18137310445308685
Epoch: 11
1/1 [==============================] - 0s 4ms/step
0.17570093274116516
Epoch: 12
1/1 [=======================